# Import Libraries

In [1]:
import os
import cv2
import random
from pathlib import Path
from tqdm import tqdm
import shutil
import numpy as np

# Define Paths

In [4]:
hr_dir = Path("C:/Users/manas/image-sharpness/data/train/HR")
lr_dir = Path("C:/Users/manas/image-sharpness/data/train/LR")
lr_patches_dir1 = Path("C:/Users/manas/image-sharpness/data/patches/LR_patches1")
hr_patches_dir = Path("C:/Users/manas/image-sharpness/data/patches/HR_patches")
lr_patches_dir= Path("C:/Users/manas/image-sharpness/data/patches/LR_patches")
patch_size = 192
scale = 4

In [3]:
print(hr_dir.resolve())
print(hr_dir.exists())
print(len(list(hr_dir.glob("*"))))

C:\Users\manas\image-sharpness\data\train\HR
True
800


# Ensure output directories exist

In [5]:
hr_patches_dir.mkdir(parents=True, exist_ok=True)
lr_patches_dir.mkdir(parents=True, exist_ok=True)

# Function to crop HR images to patches

In [6]:
def crop_image_to_patches(img, patch_size):
    h, w = img.shape[:2]
    patches = []
    for y in range(0, h - patch_size + 1, patch_size):
        for x in range(0, w - patch_size + 1, patch_size):
            patch = img[y:y + patch_size, x:x + patch_size]
            patches.append(patch)
    return patches

# generate and save all HR patches

In [8]:
all_hr_patches = []
image_files = sorted(hr_dir.glob("*.png"))

print("Generating HR patches...")
for img_path in tqdm(image_files):
    hr_img = cv2.imread(str(img_path))
    if hr_img is None:
        continue
    patches = crop_image_to_patches(hr_img, patch_size)
    for idx, patch in enumerate(patches):
        patch_name = f"{img_path.stem}_{idx:03d}.png"
        patch_path = hr_patches_dir / patch_name
        cv2.imwrite(str(patch_path), patch)
        all_hr_patches.append(patch_path)


Generating HR patches...


100%|██████████| 800/800 [04:01<00:00,  3.31it/s]


In [9]:
all_hr_patches

[WindowsPath('C:/Users/manas/image-sharpness/data/patches/HR_patches/0001_000.png'),
 WindowsPath('C:/Users/manas/image-sharpness/data/patches/HR_patches/0001_001.png'),
 WindowsPath('C:/Users/manas/image-sharpness/data/patches/HR_patches/0001_002.png'),
 WindowsPath('C:/Users/manas/image-sharpness/data/patches/HR_patches/0001_003.png'),
 WindowsPath('C:/Users/manas/image-sharpness/data/patches/HR_patches/0001_004.png'),
 WindowsPath('C:/Users/manas/image-sharpness/data/patches/HR_patches/0001_005.png'),
 WindowsPath('C:/Users/manas/image-sharpness/data/patches/HR_patches/0001_006.png'),
 WindowsPath('C:/Users/manas/image-sharpness/data/patches/HR_patches/0001_007.png'),
 WindowsPath('C:/Users/manas/image-sharpness/data/patches/HR_patches/0001_008.png'),
 WindowsPath('C:/Users/manas/image-sharpness/data/patches/HR_patches/0001_009.png'),
 WindowsPath('C:/Users/manas/image-sharpness/data/patches/HR_patches/0001_010.png'),
 WindowsPath('C:/Users/manas/image-sharpness/data/patches/HR_patc

# Shuffle patches and assign interpolation methods

In [10]:
print(" Shuffling and assigning interpolation methods...")
random.shuffle(all_hr_patches)
half = len(all_hr_patches) // 2
interp_map = {
    "bicubic": cv2.INTER_CUBIC,
    "bilinear": cv2.INTER_LINEAR
}
interp_methods = [interp_map["bicubic"]] * half + [interp_map["bilinear"]] * (len(all_hr_patches) - half)

print("Generating LR patches...")
for patch_path, interp in tqdm(zip(all_hr_patches, interp_methods), total=len(all_hr_patches)):
    hr_patch = cv2.imread(str(patch_path))
    if hr_patch is None:
        continue

    lr_patch = cv2.resize(hr_patch, (patch_size // scale, patch_size // scale), interpolation=interp)
    lr_patch_up = cv2.resize(lr_patch, (patch_size, patch_size), interpolation = interp)

    lr_path = lr_patches_dir / patch_path.name
    cv2.imwrite(str(lr_path), lr_patch_up)

print("All HR and LR patches generated and saved.")

 Shuffling and assigning interpolation methods...
Generating LR patches...


100%|██████████| 55510/55510 [14:37<00:00, 63.26it/s]

All HR and LR patches generated and saved.


## Shuffling the data and creating LR patches with Half bilinear and half bicubic with varying scale (2x,3x and 4x) for better generalizibility

In [12]:
lr_patches_dir= Path("C:/Users/manas/image-sharpness/data/patches/mixed_LR_patches")
lr_patches_dir.mkdir(parents=True, exist_ok=True)
print(" Shuffling and assigning interpolation methods...")
random.shuffle(all_hr_patches)
half = len(all_hr_patches) // 2
interp_map = {
    "bicubic": cv2.INTER_CUBIC,
    "bilinear": cv2.INTER_LINEAR
}

bicubic_patches = all_hr_patches[:half]
bilinear_patches = all_hr_patches[half:]

def assign_scales(patch_list):
    count = len(patch_list)
    third = count // 3
    remainder = count - 3 * third  # Handle leftovers
    scales = [2] * third + [3] * third + [4] * third + [4] * remainder
    random.shuffle(scales)
    return scales

bicubic_scales = assign_scales(bicubic_patches)
bilinear_scales = assign_scales(bilinear_patches)

combined = list(zip(bicubic_patches, [interp_map["bicubic"]] * len(bicubic_patches), bicubic_scales)) + \
           list(zip(bilinear_patches, [interp_map["bilinear"]] * len(bilinear_patches), bilinear_scales))

print("🔄 Shuffling all patch-interp-scale combinations...")
random.shuffle(combined)

print("⚙️ Generating LR patches with 2x/3x/4x down-up using both interpolations...")
for patch_path, interp, scale in tqdm(combined):
    hr_patch = cv2.imread(str(patch_path))
    if hr_patch is None:
        continue

    down_size = patch_size // scale
    lr_patch = cv2.resize(hr_patch, (down_size, down_size), interpolation=interp)
    lr_patch_up = cv2.resize(lr_patch, (patch_size, patch_size), interpolation=interp)

    # Save
    lr_path = lr_patches_dir / patch_path.name
    cv2.imwrite(str(lr_path), lr_patch_up)

print("✅ All mixed LR patches saved in:", lr_patches_dir)

 Shuffling and assigning interpolation methods...
🔄 Shuffling all patch-interp-scale combinations...
⚙️ Generating LR patches with 2x/3x/4x down-up using both interpolations...


100%|██████████| 55510/55510 [03:22<00:00, 274.11it/s]

✅ All mixed LR patches saved in: C:\Users\manas\image-sharpness\data\patches\mixed_LR_patches


In [ ]:
import torch

if torch.cuda.is_available():
    print("CUDA is available! Using GPU.")
    print("Current device:", torch.cuda.current_device())
    print("Device name:", torch.cuda.get_device_name(torch.cuda.current_device()))
else:
    print("CUDA is not available. Using CPU.")

In [4]:
lr_dir.mkdir(parents=True, exist_ok=True)

In [5]:
scales = [1.5, 1.7, 2.0]
interpolations = {
    'bicubic': cv2.INTER_CUBIC,
    'bilinear': cv2.INTER_LINEAR
}

In [6]:
all_hr_images = sorted(list(hr_dir.glob("*.png")))
random.shuffle(all_hr_images)


In [7]:
half = len(all_hr_images) // 2
bicubic_images = all_hr_images[:half]
bilinear_images = all_hr_images[half:]


In [8]:
def assign_scales(image_list):
    count = len(image_list)
    third = count // 3
    remainder = count - 3 * third
    scales = [1.5] * third + [1.7] * third + [2.0] * third + [2.0] * remainder
    random.shuffle(scales)
    return scales


In [9]:
bicubic_scales = assign_scales(bicubic_images)
bilinear_scales = assign_scales(bilinear_images)

In [12]:
combined = list(zip(bicubic_images, [interpolations["bicubic"]] * len(bicubic_images), bicubic_scales)) + \
           list(zip(bilinear_images, [interpolations["bilinear"]] * len(bilinear_images), bilinear_scales))

In [13]:
print(" Shuffling all image-interp-scale combinations...")
random.shuffle(combined)

print(" Generating LR images using 1.5x, 1.7x, 2x down-up scales...")
for img_path, interp, scale in tqdm(combined):
    hr_img = cv2.imread(str(img_path))
    if hr_img is None:
        continue

    h, w = hr_img.shape[:2]
    down_w, down_h = int(w / scale), int(h / scale)
    lr_img = cv2.resize(hr_img, (down_w, down_h), interpolation=interp)
    lr_img_up = cv2.resize(lr_img, (w, h), interpolation=interp)

    # Save to train/LR/
    out_path = lr_dir / img_path.name
    cv2.imwrite(str(out_path), lr_img_up)

print(" All mixed LR images saved in:", lr_dir)

 Shuffling all image-interp-scale combinations...
 Generating LR images using 1.5x, 1.7x, 2x down-up scales...


100%|██████████| 800/800 [04:29<00:00,  2.97it/s]

 All mixed LR images saved in: C:\Users\manas\image-sharpness\data\train\LR


In [5]:
lr_patches_dir1.mkdir(parents=True, exist_ok=True)

In [6]:
scales = [1.5, 1.7, 2.0]
interpolations = {
    'bicubic': cv2.INTER_CUBIC,
    'bilinear': cv2.INTER_LINEAR
}

In [7]:
all_hr_patches = sorted(list(hr_patches_dir.glob("*.png")))
all_hr_patches

[WindowsPath('C:/Users/manas/image-sharpness/data/patches/HR_patches/0001_000.png'),
 WindowsPath('C:/Users/manas/image-sharpness/data/patches/HR_patches/0001_001.png'),
 WindowsPath('C:/Users/manas/image-sharpness/data/patches/HR_patches/0001_002.png'),
 WindowsPath('C:/Users/manas/image-sharpness/data/patches/HR_patches/0001_003.png'),
 WindowsPath('C:/Users/manas/image-sharpness/data/patches/HR_patches/0001_004.png'),
 WindowsPath('C:/Users/manas/image-sharpness/data/patches/HR_patches/0001_005.png'),
 WindowsPath('C:/Users/manas/image-sharpness/data/patches/HR_patches/0001_006.png'),
 WindowsPath('C:/Users/manas/image-sharpness/data/patches/HR_patches/0001_007.png'),
 WindowsPath('C:/Users/manas/image-sharpness/data/patches/HR_patches/0001_008.png'),
 WindowsPath('C:/Users/manas/image-sharpness/data/patches/HR_patches/0001_009.png'),
 WindowsPath('C:/Users/manas/image-sharpness/data/patches/HR_patches/0001_010.png'),
 WindowsPath('C:/Users/manas/image-sharpness/data/patches/HR_patc

In [8]:
random.shuffle(all_hr_patches)

In [9]:
half = len(all_hr_patches) // 2
bicubic_images = all_hr_patches[:half]
bilinear_images = all_hr_patches[half:]

In [10]:
def assign_scales(image_list):
    count = len(image_list)
    third = count // 3
    remainder = count - 3 * third
    scales = [1.5] * third + [1.7] * third + [2.0] * third + [2.0] * remainder
    random.shuffle(scales)
    return scales

In [11]:
bicubic_scales = assign_scales(bicubic_images)
bilinear_scales = assign_scales(bilinear_images)

In [12]:
combined = list(zip(bicubic_images, [interpolations["bicubic"]] * len(bicubic_images), bicubic_scales)) + \
           list(zip(bilinear_images, [interpolations["bilinear"]] * len(bilinear_images), bilinear_scales))

In [13]:
print(" Shuffling all image-interp-scale combinations...")
random.shuffle(combined)

print(" Generating LR patches images using 1.5x, 1.7x, 2x down-up scales...")
for img_path, interp, scale in tqdm(combined):
    hr_img = cv2.imread(str(img_path))
    if hr_img is None:
        continue

    h, w = hr_img.shape[:2]
    down_w, down_h = int(w / scale), int(h / scale)
    lr_img = cv2.resize(hr_img, (down_w, down_h), interpolation=interp)
    lr_img_up = cv2.resize(lr_img, (w, h), interpolation=interp)

    # Save to train/LR/
    out_path = lr_patches_dir1 / img_path.name
    cv2.imwrite(str(out_path), lr_img_up)

print(" All mixed LR images saved in:", lr_patches_dir1)

 Shuffling all image-interp-scale combinations...
 Generating LR patches images using 1.5x, 1.7x, 2x down-up scales...


100%|██████████| 55510/55510 [07:42<00:00, 120.01it/s]

 All mixed LR images saved in: C:\Users\manas\image-sharpness\data\patches\LR_patches1


In [15]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

NameError: name 'torch' is not defined